## Task 1:

Study how basic reference counting works.
Create a string and examine its reference count at different stages.

Return a tuple: `(count_after_creation, count_after_ref, count_after_del)`


In [ ]:
import sys
import gc


def task1():
    s = "Ping"

    # 1. reference count right after creation
    count_after_creation = sys.getrefcount(s)

    # 2. create an additional reference to the same object
    other_ref = s
    count_after_ref = sys.getrefcount(s)

    # 3. remove the additional reference
    del other_ref
    gc.collect()

    # 4. reference count after deleting the extra reference
    count_after_del = sys.getrefcount(s)

    return (count_after_creation, count_after_ref, count_after_del)



print("Task 1:", task1())
# print("Once again, Task 1:", task1())

Task 1: (5, 6, 5)
Once again, Task 1: (5, 6, 5)


## Task 2: 

Effect of functions on Reference Counting

Create a list and pass it into a function. Compare its reference count before, during, and after the function call.

Return a tuple:
`(count_before_call, count_during_call, count_after_call)`

In [ ]:
import sys


def task2(lista: list = [1, 2, 3]) -> tuple[int, int, int]:

    def process_list(lista) -> int:
        # 2.1. Reference count *inside* function call
        return sys.getrefcount(lista)

    return (
        # 1. Reference count BEFORE function call:
        sys.getrefcount(lista),

        # 2. Reference count DURING function call:
        process_list(lista),

        # 3. Reference count AFTER function call:
        sys.getrefcount(lista),
    )



# Check
print("Task 2:", task2())


Task 2: (3, 4, 3)


## Task 3: 

Cyclic references and Memory Leaks

Create two objects with a cyclic reference, then break it.
Use gc to check how many objects were collected.

Return the number of objects collected by gc after breaking the reference.

In [22]:
import gc

def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    # Clean up anything pending from earlier, so the count is clearer
    gc.collect()

    # 1–3: create a cycle
    a = Node("A")
    b = Node("B")
    a.ref = b
    b.ref = a

    # 4: break external references (the cycle itself still exists)
    a = None
    b = None

    # 5: force garbage collection of the now-unreachable cycle
    collected = gc.collect()

    return collected


# Check
print("Task 3:", task3())


Task 3: 2


## Task 4: 

Comparing Reference Count for Different Data Types

Compare the reference count for numbers, strings, lists, and dictionaries.
Return a dictionary containing the reference count for each type after creation.

In [23]:
import sys

def task4():
    # different types
    a_int = 42
    a_str = "hello"
    a_list = [1, 2, 3]
    a_dict = {"a": 1, "b": 2}

    # collect reference counts
    result = {
        "int":  sys.getrefcount(a_int),
        "str":  sys.getrefcount(a_str),
        "list": sys.getrefcount(a_list),
        "dict": sys.getrefcount(a_dict)
    }

    return result


# Check
print("Task 4:", task4())


Task 4: {'int': 1000000037, 'str': 7, 'list': 2, 'dict': 2}


## Task 5: 

Weak References

Create two objects with a weak reference between them (weakref).
Verify that the objects can be garbage-collected even though a weak reference exists.
Return True if the weak reference does not increase the reference count.


In [24]:
import sys
import weakref
import gc

def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    # 1. Create object
    d = Data(10)

    # 2. Reference count before weakref
    count_before = sys.getrefcount(d)

    # 3. Create weak reference
    w = weakref.ref(d)

    # 4. Reference count after weakref (should be the same)
    count_after = sys.getrefcount(d)

    # 5. Remove strong reference
    d = None

    # 6. Run garbage collector
    gc.collect()

    # 7. Weakref should now be dead: w() returns None
    collected = (w() is None)

    # 8. Weakref must NOT increase refcount
    weakref_ok = (count_before == count_after)

    return weakref_ok and collected


# Check
print("Task 5:", task5())


Task 5: True


## Task 6: 

Monitoring the work of the Garbage Collector

Register a callback function to track GC (garbage collector) activity.

Return a list of events that the callback has detected.


In [31]:
import gc

def task6():
    events = []

    # Define callback for GC activity
    def callback(phase, info):
        events.append((phase, dict(info)))

    # Register callback
    gc.callbacks.append(callback)

    # Trigger GC manually a few times
    gc.collect()
    gc.collect()

    # Unregister callback
    gc.callbacks.remove(callback)

    return events


# Check
response = task6()
print("Task 6:")
for r in response:
    print(r)

Task 6:
('start', {'generation': 2, 'collected': 0, 'uncollectable': 0})
('stop', {'generation': 2, 'collected': 0, 'uncollectable': 0})
('start', {'generation': 2, 'collected': 0, 'uncollectable': 0})
('stop', {'generation': 2, 'collected': 0, 'uncollectable': 0})


## Task 7: 

GC Generation Analysis

Create several objects and track how they move between GC generations.

Return a tuple with the number of objects in each generation before and after object creation.

In [38]:
import gc

def task7():
    # Force a clean GC state
    gc.collect()
    before = gc.get_count()  # (gen0, gen1, gen2)

    # Create many objects to populate generation 0
    objs = [object() for _ in range(10000)]

    # Run GC so some objects move to later generations
    gc.collect()
    after = gc.get_count()

    return before, after


# Check
print("Задача 7:", task7())


Задача 7: ((22, 0, 0), (23, 0, 0))


## Task 8: 

Monitoring garbage-collection thresholds

Study how the generation counters of the GC change when creating objects.

Return a dictionary with the state of the counters before and after creating objects.

The template for the return value is:
```python
return {
    'thresholds': initial_threshold,
    'initial_count': initial_count,
    'after_creation': after_creation_count,
    'after_collect_gen0': after_collect_count,
    'collected_objects': collected
}
```

In [43]:
import gc

def task8():
    # Current thresholds for generations (when GC is triggered)
    initial_threshold = gc.get_threshold()

    # Clean up to start from a "stable" point
    gc.collect()
    initial_count = gc.get_count()          # (gen0, gen1, gen2)

    # Create many temporary objects to bump generation 0 counters
    objs = [object() for _ in range(100000)]

    # Counters after object creation
    after_creation_count = gc.get_count()

    # Run collection only for generation 0
    collected = gc.collect(0)

    # Counters after explicit GC of gen 0
    after_collect_count = gc.get_count()

    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }


response = task8()
print("Task 8:")
for k, v in response.items():
    print(f" -> {k}: {v}")

Task 8:
 -> thresholds: (700, 10, 10)
 -> initial_count: (22, 0, 0)
 -> after_creation: (40, 0, 0)
 -> after_collect_gen0: (0, 1, 0)
 -> collected_objects: 0


## Task 9:

Quick test of GC cyclic reference cleanup.

Check whether the garbage collector cleans up cyclic references.

Create a fast cycle, run GC, and return a tuple: `(count_before, count_after, collected_objects).`

In [44]:
import gc

def task9():
    # Create a fast cycle: a → b → c → a
    a, b, c = [], [], []
    a.append(b)
    b.append(c)
    c.append(a)

    # Count objects before collection
    count_before = len(gc.get_objects())

    # Force garbage collection
    collected = gc.collect()

    # Count objects after collection
    count_after = len(gc.get_objects())

    return count_before, count_after, collected

print("Task 9:", task9())


Task 9: (76456, 76455, 0)


## Task 10:

Detector of “immortal” (uncollectable) objects

Find objects that survive forced garbage collection.

Output the number of such objects.

`Hint`: objects with a __del__ method involved in a cycle cannot be collected.


In [ ]:
import gc

def task10():
    gc.collect()
    gc.set_debug(gc.DEBUG_SAVEALL)

    class Persistent:
        def __del__(self):
            pass

    # Create an uncollectable cycle
    a = Persistent()
    b = Persistent()
    a.ref = b
    b.ref = a

    # Remove external references
    del a, b

    # Force GC
    gc.collect()

    # Uncollectable objects are stored in gc.garbage
    result = len(gc.garbage)

    # Clean up for next tasks
    gc.garbage.clear()
    gc.set_debug(0)

    return result

print("Task 10:", task10())


Task 10: 2
